# CS5246 Project - Five Guys

Demo of the suggested solution.

In [1]:
%load_ext autoreload
%autoreload 2

## Step 0: Data loading

In [2]:
from simplertimes import data

# Display information on the dataset
data.describe_cnn_dm_dataset()

c:\Users\hansg\anaconda3\envs\nus_cs5246_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\hansg\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hansg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hansg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CNN/DailyMail non-anonymized summarization dataset.

There are two features:
  - article: text of news article, used as the document to be summarized
  - highlights: joined text of highlights with <s> and </s> around each
    highlight, which is the target summary


Features: {'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}

Split: ['train: 287113 samples 1.18 GB', 'validation: 13368 samples 0.05 GB', 'test: 11490 samples 0.05 GB']


In [3]:
# Get a few samples of the test dataset
three_samples = data.load_cnn_dm_dataset(split='test')[:3]
for idx in range(3):
    print(f"{idx+1}:\n    article: {three_samples['article'][idx]}\n    highlight: {three_samples['highlights'][idx]}")

2023-04-10 10:28:17 | WARNING | datasets.builder | Found cached dataset cnn_dailymail (C:/Users/hansg/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


1:
    article: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesda

## Step 1: Summarize

In [4]:
from simplertimes import summarize

# Create a BART model
bart_summarizer = summarize.create_summarizer(summarize.BART_MODEL_ID)
!nvidia-smi

Summarizer facebook/bart-large-cnn will run on device cuda:0.
Mon Apr 10 10:28:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0    N/A /  N/A |   1604MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                          

In [5]:
bart_summarizer.print_details()

            BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [6]:
# Perform inference using bart
bart_summaries = bart_summarizer.generate_summary(three_samples["article"])

# Remove BART model from memory
del bart_summarizer
import torch
torch.cuda.empty_cache()
!nvidia-smi

Mon Apr 10 10:28:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P0    N/A /  N/A |    466MiB /  4096MiB |     78%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Create a PEGASUS model
peg_summarizer = summarize.create_summarizer(summarize.PEGASUS_MODEL_ID)
!nvidia-smi

Summarizer google/pegasus-cnn_dailymail will run on device cuda:0.
Mon Apr 10 10:30:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |   2654MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                     

In [8]:
peg_summarizer.print_details()

            PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_laye

In [11]:
# Perform inference using PEGASUS
peg_summaries = peg_summarizer.generate_summary(three_samples["article"])

# Remove PEGASUS model from memory
del peg_summarizer
import torch
torch.cuda.empty_cache()
!nvidia-smi

Mon Apr 10 10:31:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   64C    P0    N/A /  N/A |    466MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
# Compare generated summaries
for idx in range(3):
    print(f"{idx+1}:\n    Article: {three_samples['article'][idx]}\n    GT Summary: {three_samples['highlights'][idx]}\n    BART: {bart_summaries[idx]['summary_text']}\n   PEGASUS: {peg_summaries[idx]['summary_text']}")

1:
    Article: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesda

## Step 2: Simplify

In [17]:
from simplertimes import simplify

# Create the ACCESS simplifier model
access_simplifier = simplify.create_simplifier()
!nvidia-smi

Mon Apr 10 10:34:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |    858MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
access_simplifier.print_details()

<function get_fairseq_simplifier.<locals>.fairseq_simplifier at 0x00000121B61AEA60>            


In [19]:
# Perform simplification using ACCESS
# Use the pegasus summary as an example
to_simplify = [ val['summary_text'] for val in peg_summaries ]
simplified, comp_simp_pairs = access_simplifier.simplify_document(to_simplify)

2023-04-10 10:35:03 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [20]:
# Nicely formatted output
for doc_idx in range(len(to_simplify)):
    print(f"============================================================")
    print()

    doc_simp = simplified[doc_idx]
    print(f"Summary:\n{doc_simp}\n")
    
    doc_sent_pairs = comp_simp_pairs[doc_idx]
    print(f"Simplifications:")
    for comp, simp in doc_sent_pairs:
        print(f"{comp}")
        print(f"--> {simp}")
    print()
    print(f"============================================================")


Summary:
The Palestinian Authority is called the 123rd member of the International Criminal Court. The move gives the court control over alleged crimes in Palestinian territories. Palestinians signed the ICC's first university in Rome Statute in January.

Simplifications:
The Palestinian Authority formally becomes the 123rd member of the International Criminal Court.
--> The Palestinian Authority is called the 123rd member of the International Criminal Court.
The move gives the court jurisdiction over alleged crimes in Palestinian territories.
--> The move gives the court control over alleged crimes in Palestinian territories.
Palestinians signed the ICC's founding Rome Statute in January.
--> Palestinians signed the ICC's first university in Rome Statute in January.


Summary:
Theia, a white-and-black bully breed mix, had many hits by a car and buried in a field. Four days later, the dog goes to a farm and is taken in by a worker. She needs surgery to get a dislocated jaw and a caved

## Step 3: Generate Vocabulary List